In [ ]:
## Do
# plot contribution to j0rad integral --> rewrite j0_rad function
# modify V_loss_Rau outputs, so it gives all voltages (Eref,Voc_SQ,Voc_jsc,Voc_rad,Voc) and all Deltas (Delta_SQ,Delta_jsc,Deta_rad,Delta_nonrad)
#find AM1.5 file difficult

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.optimize import curve_fit
from scipy import integrate as ig
from scipy.interpolate import interp1d
from functions import J0, J0_rad, SQ, Voc_rad, LED_QE, calculate_summary, Vloss_CT, Vloss_SQ, calculate_Jsc
from utils import bb, getAM15, linear, set_up_plot
from utils_EQE import extend_EQE

In [ ]:
T = 293 # [K] ambient & cell temperature
h = 6.626 * 10**(-34) # [kgm^2/s]
h_eV = 4.1357*10**(-15) # eV s
c = 2.998 * 10**(8) # [m/s]
k = 1.3806 * 10**(-23) # [kgm^2/s^2K]
k_eV = 8.6173*10**(-5) # eV / K
q = 1.60217662 * 10**(-19) # [C]
q_eV = 1
Vth = k_eV*T/q_eV # thermal voltage [eV]

In [ ]:
codePath = os.getcwd()#only makes sense after kernel restart
codePath

In [ ]:
os.chdir('C:/Users/pasca/OneDrive - Nexus365/Post/data/')#/home/jungbluth/Desktop/
os.getcwd()

### additional functions

In [ ]:
# Radiative limit of saturation current density
def J0_rad_alt(EQE_df, phi_bb_df):
    """
    Function to calculate the radiative limit of the saturation current density (J0,rad)
    :param EQE_df: dataFrame of EQE spectra, with columns 'Energy' and 'EQE' [dataFrame of floats]
    :param phi_bb_df: dataFrame of black-body spectrum, with columns 'Energy' and 'Phi' [dataFrame of floats]
    :return: J0_rad: radiative limit of the saturation current density [float]
    """
    
    #EQE_intp = interp1d(EQE_df['Energy'].values, EQE_df['EQE'].values)
    #Phi_intp = interp1d(phi_bb_df['Energy'].values, phi_bb_df['Phi'].values)

    J0_rad_integral = q/10*ig.simps(EQE_df['EQE'].values*phi_bb_df['Phi'].values,EQE_df['Energy'].values)
    #ig.simps(AM15flux[energy>=Eg],energy[energy>=Eg])
    ##plt.semilogy(EQE_df['Energy'].values,EQE_df['EQE'].values*phi_bb_df['Phi'].values)
    ##plt.semilogy(EQE_df['Energy'].values,EQE_df['EQE'].values)
    ##plt.xlim([0.8,2])
    ##plt.ylim([1e-6,1e6])
    #result = ig.quad(lambda e: q*EQE_intp(e)*Phi_intp(e), min(EQE_df['Energy']), max(EQE_df['Energy']))
    #J0_rad_integral = result[0]/10 # result[0] = integral result, result[1] = estimate of the absolute error on the result
    return J0_rad_integral

In [ ]:
#def getAM15():
    #filePath = codePath + '\ASTMG173.csv'# 1:wavelength [nm], 3:AM15g spectral power [W/(m^2nm)]
    

In [ ]:
#loading flips the data!
##VTE
#binFile = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\mobombination\Vloss\bins_Vloss_sel-fitted.csv'
##NFA bilayer
binFile = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\bilayer_solution\DataAnalysis\Vloss\bins_Vloss_stitched.csv'
df = pd.read_csv(binFile, sep = ',') #uses .csv headers as column labels 
#row = df.loc[9]#thin2,4,8 #thick3,5,9
row = df.loc[3]
EQE_file = row['EQE_folder'] + '\\' + row['EQE_file']
EQEdata = np.genfromtxt(EQE_file, skip_header = 1, delimiter = ',')
EQE_df = pd.DataFrame()
EQE_df['Energy'] = np.flip(EQEdata[:,2])
EQE_df['EQE'] = np.flip(EQEdata[:,3])*row['EQE_fac']
os.chdir(codePath) 
phi_bb_df = bb(EQE_df['Energy'])

Jsc_EQE = calculate_Jsc(EQE_df['Energy'],EQE_df['EQE'])
J0rad_EQE_alt = J0_rad_alt(EQE_df, phi_bb_df)
Voc_rad_alt = k*T/q * np.log((Jsc_EQE/J0rad_EQE_alt )+1)

#cumulative sum (contribution to integral, lower boundary varied for *_rev) 
J0_rad_cum_rev = -q/10*ig.cumulative_trapezoid(np.flip(EQE_df['EQE'].values*phi_bb_df['Phi'].values),np.flip(EQE_df['Energy'].values))
Voc_rad_cum = k*T/q * np.log((Jsc_EQE/J0_rad_cum_rev )+1)
####J0_rad_cum = q/10*ig.cumulative_trapezoid(EQE_df['EQE'].values*phi_bb_df['Phi'].values,EQE_df['Energy'].values)
##

In [ ]:
df_Vocrad = pd.DataFrame()
df_Vocrad['Emin'] = np.flip(EQE_df.loc[1:,'Energy'].values)
df_Vocrad['Vocrad'] = Voc_rad_cum
save_dir = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\mobombination\Vloss\res'
save_path = os.path.join(save_dir, 'Vocrad_contrib_' + row['run'] + '.csv')
df_Vocrad.to_csv(save_path)

In [ ]:
## plot Vocrad contributions
print(J0rad_EQE_alt )
print(Voc_rad_alt)

plt.figure(1)
plt.semilogy(EQE_df['Energy'].values, EQE_df['EQE'].values)
plt.semilogy(EQE_df['Energy'].values, EQE_df['EQE'].values*phi_bb_df['Phi'].values)
plt.ylim([1e-8,1e6])
plt.xlim([0.6,1.8])
plt.ylabel('EQE, integrand = EQE*phi_bb')
plt.xlabel('energy E [eV]')

plt.figure(2)
plt.semilogy(np.flip(EQE_df.loc[1:,'Energy'].values),J0_rad_cum_rev)
plt.xlim([0.5,1.8])
plt.ylim([1e-26,1e-14])
plt.ylabel('J0_rad')
plt.xlabel('lower boundary of integral E_min [eV]')

plt.figure(3)
plt.plot(np.flip(EQE_df.loc[1:,'Energy'].values),Voc_rad_cum)
plt.xlim([0.6,1.4])
plt.ylim([0.8,1.5])
plt.ylabel('Voc_rad')
plt.xlabel('lower boundary of integral E_min [eV]')

In [ ]:
#loading doesn't flip the data!
binFile = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\mobombination\Vloss\bins_Vloss_extrapolate.csv'
df = pd.read_csv(binFile, sep = ',') #uses .csv headers as column labels 
row = df.loc[0]
EQE_file = row['EQE_folder'] + '\\' + row['EQE_file']
EQEdata = np.genfromtxt(EQE_file, skip_header = 1, delimiter = ',')
EQE_df = pd.DataFrame()
EQE_df['Energy'] = EQEdata[:,2]
EQE_df['EQE'] = EQEdata[:,3]*row['EQE_fac']
os.chdir(codePath) 
phi_bb_df = bb(EQE_df['Energy'])

In [ ]:
#'linear' extrapolation --> not successful
EQE_extrap = extend_EQE(EQE_df, 'linear', min_energy=0.8, start = 1.3, stop = 1.4)
phi_bb_df_etrap = bb(EQE_extrap['Energy'])
#plt.figure(11)
#plt.plot(EQE_extrap['Energy'])
#plt.figure(12)
#plt.semilogy(EQE_extrap['EQE'])
plt.figure(10)
plt.semilogy(EQE_extrap['Energy'],EQE_extrap['EQE'])

In [ ]:
# 'gaussian' extrapolation
## equivalent to sitiching fit & data

In [ ]:
plt.figure(0)
plt.semilogy(EQE_df['Energy'].values, EQE_df['EQE'].values)
plt.semilogy(EQE_df['Energy'].values, EQE_df['EQE'].values*phi_bb_df['Phi'].values)
###
plt.semilogy(EQE_extrap['Energy'].values, EQE_extrap['EQE'].values)
plt.semilogy(EQE_extrap['Energy'].values, EQE_extrap['EQE'].values*phi_bb_df_etrap['Phi'].values)
plt.ylim([1e-14,1e6])
plt.xlim([0.5,2])
plt.ylabel('EQE, integrand = EQE*phi_bb')
plt.xlabel('energy E [eV]')

### Calculate Relevant Parameters

In [ ]:
##VTE
#binFile = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\mobombination\Vloss\bins_Vloss_sel-fitted.csv'
##NFA bilayer
binFile = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\bilayer_solution\DataAnalysis\Vloss\bins_Vloss_stitched.csv'
df = pd.read_csv(binFile, sep = ',') #uses .csv headers as column labels    
df['Voc_SQ']=np.nan
df['Voc_sc']=np.nan
df['Voc_rad']=np.nan
##
#df['Voc_rad_alt']=np.nan
##
df['Delta_V_SQ'] = np.nan
df['Delta_V_sc']=np.nan
df['Delta_V_rad']=np.nan
df['Delta_V_nonrad']=np.nan
df['Q_LED']=np.nan

for ind, row in df.iterrows():
    EQE_file = row['EQE_folder'] + '\\' + row['EQE_file']
    EQEdata = np.genfromtxt(EQE_file, skip_header = 1, delimiter = ',')
    EQE_df = pd.DataFrame()
    EQE_df['Energy'] = np.flip(EQEdata[:,2])
    EQE_df['EQE'] = np.flip(EQEdata[:,3])*row['EQE_fac']
    os.chdir(codePath)  
    phi_bb_df = bb(EQE_df['Energy'])
    Jsc_EQE = calculate_Jsc(EQE_df['Energy'],EQE_df['EQE'])
    ##
    J0rad_EQE = J0_rad(EQE_df, phi_bb_df)
    vocrad,temp = Voc_rad(row['Voc'], Jsc_EQE, J0rad_EQE)
    df.at[ind,'Voc_rad'] = vocrad
    
    #J0rad_EQE_alt = J0_rad_alt(EQE_df, phi_bb_df)
    #vocrad,temp = Voc_rad(row['Voc'], Jsc_EQE, J0rad_EQE_alt)
    #df.at[ind,'Voc_rad_alt'] = vocrad        
    ##
    df.at[ind,'Voc_SQ'], df.at[ind,'Delta_V_sc'], df.at[ind,'Delta_V_rad'], df.at[ind,'Delta_V_nonrad'] = Vloss_SQ(row['Eg'], row['Voc'], row['Jsc'], voc_rad = vocrad)#
df['Voc_sc'] = df['Voc_SQ'] - df['Delta_V_sc']
df['Delta_V_SQ'] = df['Eg'] - df['Voc_SQ']
for ind, row in df.iterrows():
    df.at[ind,'Q_LED'] = LED_QE( row['Delta_V_nonrad'])

In [ ]:
df

In [ ]:
###VTE
#save_dir = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\mobombination\Vloss\res'
##NFA bilayer
save_dir = r'C:\Users\pasca\OneDrive - Nexus365\Post\projects\_MY-SAMPLES\bilayer_solution\DataAnalysis\Vloss\res'
save_path = os.path.join(save_dir, 'VlossAnalysis_new.csv')
df.to_csv(save_path)
 